In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Define a custom MAPE function to handle potential division by zero
def calculate_mape(y_true, y_pred):
    non_zero_true_indices = y_true != 0
    y_true_filtered = y_true[non_zero_true_indices]
    y_pred_filtered = y_pred[non_zero_true_indices]

    if len(y_true_filtered) == 0:
        return np.nan

    return np.mean(np.abs((y_true_filtered - y_pred_filtered) / y_true_filtered)) * 100

# Create a scorer for RandomizedSearchCV
mape_scorer = make_scorer(calculate_mape, greater_is_better=False)

# --- 1. Load Data ---
df = None

file_path_excel = r"C:\Users\tinum\Downloads\LTF Challenge data with dictionary.xlsx"
sheet_name = 'TrainData'

try:
    print(f"Attempting to load data from Excel: {file_path_excel} (sheet: {sheet_name})")
    df = pd.read_excel(file_path_excel, sheet_name=sheet_name)
    print("Successfully loaded data from Excel.")
except FileNotFoundError:
    print(f"Excel file not found at the specified path: {file_path_excel}.")
    print("Please double-check the path and ensure the file exists.")
    raise RuntimeError("Failed to load data. File not found.")
except Exception as e:
    print(f"Could not load data from Excel. Error: {e}")
    print(f"Please ensure the sheet name '{sheet_name}' is correct and the Excel file is not corrupted.")
    raise RuntimeError("Failed to load data. Please check sheet name or file integrity.")

if df is None:
    raise RuntimeError("DataFrame 'df' was not loaded. Cannot proceed.")

print("\n--- Columns in the loaded DataFrame ---")
print(df.columns.tolist())
print("---------------------------------------")

Attempting to load data from Excel: C:\Users\tinum\Downloads\LTF Challenge data with dictionary.xlsx (sheet: TrainData)
Successfully loaded data from Excel.

--- Columns in the loaded DataFrame ---
['FarmerID', 'State', 'REGION', 'SEX', 'CITY', 'Zipcode', 'DISTRICT', 'VILLAGE', 'MARITAL_STATUS', 'Location', 'Address type', 'Ownership', 'No_of_Active_Loan_In_Bureau', 'Avg_Disbursement_Amount_Bureau', 'Non_Agriculture_Income', 'Total_Land_For_Agriculture', 'K022-Village category based on Agri parameters (Good, Average, Poor)', 'K022-Nearest Mandi Name', 'K022-Proximity to nearest mandi (Km)', 'K022-Proximity to nearest railway (Km)', 'KO22-Village score based on socio-economic parameters (0 to 100)', 'K022-Village category based on socio-economic parameters (Good, Average, Poor)', 'K022-Seasonal Average Rainfall (mm)', 'K022-Ambient temperature (min & max)', 'R022-Village category based on Agri parameters (Good, Average, Poor)', 'R022-Seasonal Average Rainfall (mm)', 'R022-Ambient temper

In [7]:
possible_target_col_names = ['Target_Variable/Total Income', 'Total Income', 'Target_Variable']
found_target_col = False
for col_name in possible_target_col_names:
    if col_name in df.columns:
        original_target_col_name = col_name
        df = df.rename(columns={original_target_col_name: 'Farmer_Income'})
        print(f"Renamed '{original_target_col_name}' to 'Farmer_Income'.")
        found_target_col = True
        break

if not found_target_col:
    print(f"Error: None of the expected target column names {possible_target_col_names} found in columns.")
    raise KeyError(f"Target column (expected: 'Farmer_Income', original options: {possible_target_col_names}) not found after loading data. Cannot proceed.")

if 'Label' in df.columns:
    df = df.drop(columns=['Label'])

print("\n--- Initial Data Info ---")
df.info()
print("\nMissing values before preprocessing:")
print(df.isnull().sum()[df.isnull().sum() > 0])
print("\nDescriptive statistics for numerical features:")
print(df.describe())


Renamed 'Target_Variable/Total Income' to 'Farmer_Income'.

--- Initial Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47970 entries, 0 to 47969
Columns: 105 entries, FarmerID to Farmer_Income
dtypes: float64(62), int64(5), object(38)
memory usage: 38.4+ MB

Missing values before preprocessing:
Location                                                        17030
Address type                                                    17030
Ownership                                                       17030
Avg_Disbursement_Amount_Bureau                                  20790
Total_Land_For_Agriculture                                         71
Perc_of_house_with_6plus_room                                     168
Women_15_19_Mothers_or_Pregnant_at_time_of_survey                 168
perc_of_pop_living_in_hh_electricity                              168
perc_Households_with_Pucca_House_That_Has_More_Than_3_Rooms       168
mat_roof_Metal_GI_Asbestos_sheets                        

In [9]:
# --- NEW DEBUGGING STEP: Inspect Raw Temperature Columns ---
# This is for diagnostic purposes, the parsing logic below will handle it
print("\n--- Sample of Temperature Columns Before Parsing ---")
temp_cols_raw_check = [col for col in df.columns if 'Ambient temperature (min & max)' in col]
if not temp_cols_raw_check:
    print("No 'Ambient temperature (min & max)' columns found in the raw DataFrame.")
else:
    for col in temp_cols_raw_check:
        print(f"\nColumn: {col}")
        print(df[col].value_counts(dropna=False).head(10))
        print(df[col].head())
print("----------------------------------------------------")



--- Sample of Temperature Columns Before Parsing ---

Column: K022-Ambient temperature (min & max)
K022-Ambient temperature (min & max)
23.2 /29.79     539
21.75 /28.24    423
24.5 /31.77     400
21.71 /28.4     391
22.28 /28.86    389
21.94 /28.65    384
23.21 /30.56    372
22.38 /29.15    362
23.02 /30.16    322
23.26 /30.46    308
Name: count, dtype: int64
0    23.34 /30.33
1    23.28 /29.99
2    23.25 /31.26
3       22 /28.61
4    23.37 /30.51
Name: K022-Ambient temperature (min & max), dtype: object

Column: R022-Ambient temperature (min & max)
R022-Ambient temperature (min & max)
20.14 /31.5     539
16.18 /28.07    423
15.13 /27.78    400
19.66 /31.13    391
20.55 /31.46    389
21.04 /31.67    384
15.38 /28.12    372
18.59 /30.6     362
16.88 /29.13    322
15.09 /27.63    308
Name: count, dtype: int64
0    16.19 /29.07
1     19.7 /30.52
2    15.62 /28.67
3    19.81 /31.15
4    15.65 /28.52
Name: R022-Ambient temperature (min & max), dtype: object

Column: K021-Ambient temperatur

In [13]:
 ##3. Feature Engineering and Preprocessing ---

X = df.drop('Farmer_Income', axis=1)
y = df['Farmer_Income']

# Apply log transformation to the target variable to handle skewness
# This is done *before* splitting so the original y is preserved for final MAPE calculation
y_transformed = np.log1p(y)

# Handle 'Ambient temperature (min & max)' columns - FIXED PARSING LOGIC
temp_cols_to_process = [col for col in X.columns if 'Ambient temperature (min & max)' in col]

for col in temp_cols_to_process:
    X[col] = X[col].astype(str).str.strip().replace('', np.nan).fillna(np.nan)

    parsed_min_temps = []
    parsed_max_temps = []
    for temp_range_str in X[col]:
        if pd.isna(temp_range_str) or ' /' not in temp_range_str: # FIXED: changed '-' to ' /'
            min_val, max_val = np.nan, np.nan
        else:
            try:
                min_str, max_str = temp_range_str.split(' /') # FIXED: changed '-' to ' /'
                min_val = float(min_str.strip())
                max_val = float(max_str.strip())
            except (ValueError, TypeError):
                min_val, max_val = np.nan, np.nan
        parsed_min_temps.append(min_val)
        parsed_max_temps.append(max_val)

    X[col.replace(' (min & max)', '_min_temp')] = parsed_min_temps
    X[col.replace(' (min & max)', '_max_temp')] = parsed_max_temps
    X = X.drop(columns=[col])


# Additional Feature Engineering based on dictionary insights
for year_season in ['K021', 'K022', 'R020', 'R021', 'R022']:
    min_col = f'{year_season}-Ambient temperature_min_temp'
    max_col = f'{year_season}-Ambient temperature_max_temp'
    if min_col in X.columns and max_col in X.columns:
        X[min_col] = pd.to_numeric(X[min_col], errors='coerce')
        X[max_col] = pd.to_numeric(X[max_col], errors='coerce')
        X[f'{year_season}_Avg_Ambient_Temp'] = (X[min_col] + X[max_col]) / 2


kharif_agri_score_cols_raw = [col for col in df.columns if 'Kharif Seasons' in col and 'Agricultural Score' in col]
rabi_agri_score_cols_raw = [col for col in df.columns if 'Rabi Seasons' in col and 'Agricultural Score' in col]

kharif_agri_score_cols_in_X = [col for col in kharif_agri_score_cols_raw if col in X.columns]
rabi_agri_score_cols_in_X = [col for col in rabi_agri_score_cols_raw if col in X.columns]

if kharif_agri_score_cols_in_X:
    X['Avg_Kharif_Agri_Score'] = X[kharif_agri_score_cols_in_X].mean(axis=1)
if rabi_agri_score_cols_in_X:
    X['Avg_Rabi_Agri_Score'] = X[rabi_agri_score_cols_in_X].mean(axis=1)

if 'K022-Proximity to nearest mandi (Km)' in X.columns and 'K022-Proximity to nearest railway (Km)' in X.columns:
    X['K022-Proximity to nearest mandi (Km)'] = pd.to_numeric(X['K022-Proximity to nearest mandi (Km)'], errors='coerce')
    X['K022-Proximity to nearest railway (Km)'] = pd.to_numeric(X['K022-Proximity to nearest railway (Km)'], errors='coerce')
    X['Total_Proximity'] = X['K022-Proximity to nearest mandi (Km)'] + X['K022-Proximity to nearest railway (Km)']


# Update numerical and categorical features lists after feature engineering and dropping columns
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

In [ ]:
# --- 4. Model Training and Evaluation with Hyperparameter Tuning ---

# Split data into training and testing sets (using the transformed y)
X_train, X_test, y_train_transformed, y_test_transformed = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# Also keep the original y_test for final MAPE calculation
_, _, _, y_test_original = train_test_split(X, y, test_size=0.2, random_state=42)


# Define models and their parameter distributions for RandomizedSearchCV
models_and_params = {
    'XGBoost': {
        'model': xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1),
        'params': {
            'model__n_estimators': [500, 1000, 1500],
            'model__learning_rate': [0.01, 0.05, 0.1],
            'model__max_depth': [3, 5, 7, 9],
            'model__subsample': [0.6, 0.8, 1.0],
            'model__colsample_bytree': [0.6, 0.8, 1.0],
            'model__gamma': [0, 0.1, 0.2],
            'model__reg_alpha': [0, 0.005, 0.1]
        }
    },
    'LightGBM': {
        'model': lgb.LGBMRegressor(objective='regression', random_state=42, n_jobs=-1),
        'params': {
            'model__n_estimators': [500, 1000, 1500],
            'model__learning_rate': [0.01, 0.05, 0.1],
            'model__num_leaves': [20, 31, 40, 50],
            'model__max_depth': [5, 7, 9, 11],
            'model__subsample': [0.6, 0.8, 1.0],
            'model__colsample_bytree': [0.6, 0.8, 1.0],
            'model__reg_alpha': [0, 0.005, 0.1],
            'model__reg_lambda': [0, 0.005, 0.1]
        }
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42, n_jobs=-1),
        'params': {
            'model__n_estimators': [500, 1000, 1500],
            'model__max_depth': [10, 20, 30, None],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__max_features': [0.6, 0.8, 1.0, 'sqrt']
        }
    }
}

results = {}

print("\n--- Starting Model Training and Hyperparameter Tuning (This will take time) ---")
for name, config in models_and_params.items():
    print(f"\nTraining and Tuning {name}...")
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', config['model'])]) # Generic 'model' name for tuning

    # Using RandomizedSearchCV for efficiency, adjust n_iter as needed (more = longer but potentially better)
    # n_iter=50 is a reasonable starting point, but could be increased for more thorough search
    search = RandomizedSearchCV(
        pipeline,
        config['params'],
        n_iter=20, # Number of parameter settings that are sampled. Reduce for faster runs.
        cv=KFold(n_splits=5, shuffle=True, random_state=42), # 5-fold cross-validation
        scoring=mape_scorer,
        verbose=1,
        n_jobs=-1, # Use all available cores
        random_state=42
    )
    search.fit(X_train, y_train_transformed)

    best_model = search.best_estimator_
    y_pred_transformed = best_model.predict(X_test)

    # Inverse transform predictions
    y_pred = np.expm1(y_pred_transformed)

    # Ensure no negative predictions
    y_pred[y_pred < 0] = 0

    mape = calculate_mape(y_test_original, y_pred) # Use original y_test for final MAPE
    results[name] = {
        'MAPE': mape,
        'Best Params': search.best_params_
    }
    print(f"{name} Best MAPE (from CV): {-search.best_score_:.2f}%") # Best score is negative for 'greater_is_better=False'
    print(f"{name} Test Set MAPE: {mape:.2f}%")
    print(f"{name} Best Parameters: {search.best_params_}")


--- Starting Model Training and Hyperparameter Tuning (This will take time) ---

Training and Tuning XGBoost...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
# --- 5. Comparison of Models ---
print("\n--- Model Comparison ---")
final_mape_scores = {name: res['MAPE'] for name, res in results.items()}
for name, mape in final_mape_scores.items():
    print(f"{name}: {mape:.2f}% MAPE")
    print(f"  Best Params: {results[name]['Best Params']}")

# Optional: Visualize results
model_names_for_plot = list(final_mape_scores.keys())
mape_scores_for_plot = list(final_mape_scores.items())
mape_scores_for_plot.sort(key=lambda item: item[1])
sorted_model_names = [item[0] for item in mape_scores_for_plot]
sorted_mape_scores = [item[1] for item in mape_scores_for_plot]

plt.figure(figsize=(12, 7))
sns.barplot(x=sorted_model_names, y=sorted_mape_scores, palette='viridis')
plt.ylabel('MAPE (%)', fontsize=12)
plt.title('Comparison of Tuned Model Performance (MAPE)', fontsize=14)
plt.ylim(0, max(sorted_mape_scores) * 1.2 if sorted_mape_scores else 100)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
for index, value in enumerate(sorted_mape_scores):
    plt.text(index, value + 0.5, f'{value:.2f}%', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.show()

In [1]:
# --- 6. Further Steps for Optimization ---
print("\n--- Further Optimization Steps ---")
print("1. **Increase RandomizedSearchCV Iterations**: If results are promising, increase `n_iter` in `RandomizedSearchCV` for a more thorough search, or switch to `GridSearchCV` for exhaustive search if computational resources allow.")
print("2. **Refine Hyperparameter Ranges**: Based on the 'Best Params' found, narrow down the search ranges for tuning in subsequent runs.")
print("3. **Explore More Feature Engineering**: Focus on creating more domain-specific features related to agriculture (e.g., ratios of land holding, changes in agricultural scores over years). Consider interaction terms between high-impact features.")
print("4. **Advanced Categorical Encoding**: For high-cardinality categorical features (like `CITY`, `DISTRICT`, `FarmerID` if it were to be used as a feature), explore techniques like Target Encoding or leave-one-out encoding if One-Hot Encoding creates too many features or CatBoost's native handling.")
print("5. **Outlier Detection and Treatment**: Beyond log transformation of the target, analyze outliers in numerical features and consider winsorization or removal if they are clearly erroneous.")
print("6. **Ensemble Modeling**: Consider techniques like stacking or weighted averaging of the best-performing models to further boost predictive power.")
print("7. **External Data Integration**: As mentioned in the problem statement, sourcing and integrating external data (e.g., specific crop yields, commodity prices, more detailed government schemes, localized weather anomalies) would be a significant step.")
print("8. **Deep Dive Error Analysis**: Analyze samples where the predictions have very high MAPE to understand what specific characteristics of those farmers or locations lead to large errors. This can reveal missing features or biases.")


--- Further Optimization Steps ---
1. **Increase RandomizedSearchCV Iterations**: If results are promising, increase `n_iter` in `RandomizedSearchCV` for a more thorough search, or switch to `GridSearchCV` for exhaustive search if computational resources allow.
2. **Refine Hyperparameter Ranges**: Based on the 'Best Params' found, narrow down the search ranges for tuning in subsequent runs.
3. **Explore More Feature Engineering**: Focus on creating more domain-specific features related to agriculture (e.g., ratios of land holding, changes in agricultural scores over years). Consider interaction terms between high-impact features.
4. **Advanced Categorical Encoding**: For high-cardinality categorical features (like `CITY`, `DISTRICT`, `FarmerID` if it were to be used as a feature), explore techniques like Target Encoding or leave-one-out encoding if One-Hot Encoding creates too many features or CatBoost's native handling.
5. **Outlier Detection and Treatment**: Beyond log transformatio